In [1]:
from ctypes import (
    CDLL, POINTER, c_char_p, c_bool, c_double, c_int,
    c_short, c_void_p, c_float, c_ushort, Structure,
    c_ulong, c_long, c_uint, c_uint64, byref
)
from qudi.hardware.tcspc.spc_def import (
    SPCdata, SPCModInfo, SPC_EEP_Data, SPC_Adjust_Para,
    SPCMemConfig, PhotStreamInfo, PhotInfo, PhotInfo64,
    rate_values
)
import time
from qudi.hardware.tcspc.tcspc import SPCDllWrapper
import os

In [4]:
tcspc = SPCDllWrapper()

ini_file_path = os.path.abspath(r'C:\EXP\python\Qoptics_exp\new_settings.ini')
init_status, args = tcspc.SPC_init(ini_file_path)
print(f'Init status: {init_status} with args: {args}')

module_no = 0
init_status, args = tcspc.SPC_get_init_status(module_no)
print(f'Init status of module {module_no}: {init_status} with args: {args}')

status, mode, force_use, in_use = tcspc.SPC_set_mode(0, 1, 1)
print(f'Get mode status: {status} with mode: {mode} and force_use: {force_use} and in_use: {in_use}')

init_status, args = tcspc.SPC_get_init_status(module_no)
print(f'Init status of module {module_no}: {init_status} with args: {args}')

status, mod_no, data = tcspc.SPC_get_parameters(module_no)
print(f'Get parameters status: {status} with mod_no: {mod_no} and data collect time: {data.collect_time}')

Init status: 0 with args: C:\EXP\python\Qoptics_exp\new_settings.ini
Init status of module 0: -6 with args: c_short(0)
Get mode status: 0 with mode: c_short(0) and force_use: c_short(1) and in_use: c_long(1)
Init status of module 0: 0 with args: c_short(0)
Get parameters status: 0 with mod_no: c_short(0) and data collect time: 3.0


In [7]:
status, mod_no, adc_resolution, no_of_routing_bits, mem_info = tcspc.SPC_configure_memory(module_no, 10, 3, SPCMemConfig())
print(f'Configure memory status: {status} with adc_resolution: {adc_resolution}, no_of_routing_bits: {no_of_routing_bits} and mem_info: {mem_info}')
no_of_blocks = mem_info.max_block_no

page_no = 0

status, mod_no, page = tcspc.SPC_set_page(module_no, page_no)
print(f'Set page status: {status} with mod_no: {mod_no} and page: {page}')

status, block, mod_no, page, fill_value = tcspc.SPC_fill_memory(module_no, 0, page_no, 1)
print(f'Fill memory status: {status} with block: {block}, page: {page} and fill_value: {fill_value}')

Configure memory status: 0 with adc_resolution: c_short(10), no_of_routing_bits: c_short(3) and mem_info: <qudi.hardware.TCSPC.spc_def.SPCMemConfig object at 0x00000283305F6DC0>
Set page status: 0 with mod_no: c_short(0) and page: c_long(0)
Fill memory status: 0 with block: c_short(0), page: c_long(0) and fill_value: c_ushort(1)


In [8]:
red_factor = 1
no_of_points = int(mem_info.block_length / red_factor)
print(f'Max block no: {mem_info.max_block_no}')
print(f'Blocks per frame: {mem_info.blocks_per_frame}')
print(f'Frames per page: {mem_info.frames_per_page}')
print(f'Max page: {mem_info.maxpage}')
print(f'Block length: {mem_info.block_length}')
print(f'No of points: {no_of_points}')
data_buffer = (c_ushort * no_of_points)()

Max block no: 4096
Blocks per frame: 8
Frames per page: 1
Max page: 512
Block length: 1024
No of points: 1024


In [ ]:
status, mod_no = tcspc.SPC_start_measurement(module_no)
print(f'Start measurement status: {status} with mod_no: {mod_no}')

state_var = c_short()
status, mod_no, state = tcspc.SPC_test_state(module_no, state_var)
print(f'Test state status: {status} with mod_no: {mod_no} and state: {bytes(state)}')
tcspc.translate_status(state)

print('Sleeping for a sec')
time.sleep(1)

state_var = c_short()
status, mod_no, state = tcspc.SPC_test_state(module_no, state_var)
print(f'Test state status: {status} with mod_no: {mod_no} and state: {bytes(state)}')
tcspc.translate_status(state)

status, mod_no, block, page, reduction_factor, var_from, var_to, data = tcspc.SPC_read_data_block(module_no, 0, page_no, red_factor, 0, no_of_points - 1, data_buffer)
print(f'Read data block status: {status} with mod_no: {mod_no}, block: {block}, page: {page}, reduction_factor: {reduction_factor}, var_from: {var_from}, var_to: {var_to} and data: {data}')
print(f'Data list: {list(data_buffer)}')

#plt.plot(data_buffer)
#plt.show()

: 